### This notebook is meant for the visualization of the fully connected layers

In [1]:
import os
import torch
import numpy as np
import cv2 as cv
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# My stuff
import ae
import utils

In [10]:
# Set the root path of the particle generator experiments folder
exp_root = "/media/hdd1/kai/particle_generator/experiments/"

In [14]:
# Get the names of all the experiments in the exp_root folder
exp_paths = []
for path in os.listdir(exp_root):
    exp_paths.append(os.path.join(exp_root, path))

print("-"*60)
for i in range(len(exp_paths)):
    print("\n {}:".format(str(i)), exp_paths[i], '\n')
    print("-"*60)

------------------------------------------------------------

 0: /media/hdd1/kai/particle_generator/experiments/01-06-2020_11-29-26_ae_7500_epochs_LArCV_64_dataset 

------------------------------------------------------------

 1: /media/hdd1/kai/particle_generator/experiments/01-06-2020_11-28-04_ae_7500_epochs_LArCV_128_dataset 

------------------------------------------------------------


In [4]:
# Set the dir of the particular experiment for which to load a model
exp_dir = exp_paths[0]

In [5]:
# Create the full path to the experiment
exp_path = os.path.join(exp_root, exp_dir)
print("Experiment path set as: \n{}".format(exp_path))

Experiment path set as: 
/media/hdd1/kai/particle_generator/experiments/01-05-2020_12-32-49_ae_7500_epochs_LArCV_64_dataset/


In [6]:
# Path to model weights
weights_dir = "weights/"

In [7]:
# Load the config csv as a dict
config_csv = exp_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

OSError: File b'/media/hdd1/kai/particle_generator/experiments/01-05-2020_12-32-49_ae_7500_epochs_LArCV_64_dataset/config.csv' does not exist

In [8]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].item())**2
im_dim   = int(np.sqrt(im_size))

NameError: name 'config_df' is not defined

In [9]:
# Set up AE layer sizes
base = [256] 

# Compute encoder sizes
sizes = lambda: [ (yield 2**i) for i in range(n_layers) ]
enc_sizes = base * n_layers
enc_sizes = [a*b for a,b in zip(enc_sizes, [*sizes()])][::-1]

# Update kwarg dicts
# Decoder is the reverse of the encoder
ae_kwargs = {'enc_sizes' : enc_sizes, 'l_dim' : l_dim, 'im_size' : im_size, 'dec_sizes' : enc_sizes[::-1]}

NameError: name 'n_layers' is not defined

In [ ]:
# Get checkpoint name(s)
checkpoint_path  = exp_path + weights_dir
checkpoint_names = []
for file in os.listdir(checkpoint_path):
    checkpoint_names.append(os.path.join(checkpoint_path, file))

In [ ]:
print("-"*60)
for i in range(len(checkpoint_names)):
    print("\n {} :".format(str(i)), checkpoint_names[i], '\n')
    print("-"*60)

In [ ]:
# Select the disired checkpoint from the list
best_checkpoint = checkpoint_names[0]

In [ ]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(best_checkpoint)

In [ ]:
# Load the model on GPU (GPU 1 is the best option)
device = torch.device(1)
model = ae.AutoEncoder(**ae_kwargs).to(device)

In [ ]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Put the model in evaluation mode
model.eval()

### Visualize the fully-connected layers

[PyTorch forums discussion on layerwise viz](https://discuss.pytorch.org/t/how-to-visualize-fully-connected-layers-as-images/13626/2)